In [190]:
import pandas as pd

In [192]:
stay_0901 = pd.read_csv('datas/stay_20231001_15/stay_20231007_1.csv')
stay_0901.head()

,hdong_cd,date,time,gender,age,purpose,stay_cnts
0,4115056800,20231007,15:00,1,7,0,832
1,4611074500,20231007,22:00,1,5,0,1351
2,4711158000,20231007,21:00,1,2,0,981
3,2729053500,20231007,19:00,0,3,0,939
4,4113353000,20231007,22:00,1,2,0,1248


In [193]:
len(stay_0901)

3841094

In [194]:
stay_counts = stay_0901.pivot_table(values='stay_cnts', index='hdong_cd', aggfunc='sum')
sorted_stay_counts = stay_counts.sort_values('stay_cnts', ascending=False)
sorted_stay_counts.idxmax().item()

1156054000

In [195]:
hdong = pd.read_csv('datas/hdong.csv', index_col='행정동코드')
len(hdong)

3883

In [196]:
hdong.index.rename('hdong_cd', inplace=True)

### 여행 목적을 가진 녀석들만 보기

In [225]:
stay_0901_trip = stay_0901#[stay_0901['purpose'] == 5]
stay_0901_trip


,hdong_cd,date,time,gender,age,purpose,stay_cnts
0,4115056800,20231007,15:00,1,7,0,832
1,4611074500,20231007,22:00,1,5,0,1351
2,4711158000,20231007,21:00,1,2,0,981
3,2729053500,20231007,19:00,0,3,0,939
4,4113353000,20231007,22:00,1,2,0,1248
...,...,...,...,...,...,...,...
3841089,3117051000,20231007,08:00,1,7,5,6
3841090,2717054000,20231007,23:00,1,8,4,9
3841091,4519047000,20231007,15:00,1,4,1,11
3841092,4377025000,20231007,14:00,0,8,3,7


In [226]:
stay_0901_trip_reduced =  stay_0901_trip.drop(['date', 'purpose'], axis=1).sort_values(by=['hdong_cd', 'time'])#.groupby(
#     ['hdong_cd', 'time', 'gender', 'age']
# )['stay_cnts'].sum().reset_index()

stay_0901_trip_reduced


,hdong_cd,time,gender,age,stay_cnts
18511,1111051500,08:00,0,0,646
26619,1111051500,08:00,1,3,1270
36954,1111051500,08:00,1,4,1464
47252,1111051500,08:00,1,0,700
71523,1111051500,08:00,0,2,808
...,...,...,...,...,...
3213136,5183035000,23:00,0,6,342
3222447,5183035000,23:00,1,5,343
3339016,5183035000,23:00,1,7,115
3359290,5183035000,23:00,1,4,114


In [227]:
stay_0901_trip_time = stay_0901_trip_reduced.groupby(['hdong_cd', 'time'])['stay_cnts'].sum().reset_index()
stay_0901_trip_time['diff'] = stay_0901_trip_time.stay_cnts - stay_0901_trip_time.stay_cnts.shift()
stay_0901_trip_time

,hdong_cd,time,stay_cnts,diff
0,1111051500,08:00,19621,NaN
1,1111051500,09:00,23280,3659.0
2,1111051500,10:00,27712,4432.0
3,1111051500,11:00,31436,3724.0
4,1111051500,12:00,35029,3593.0
...,...,...,...,...
56443,5183035000,19:00,12378,-164.0
56444,5183035000,20:00,12350,-28.0
56445,5183035000,21:00,12001,-349.0
56446,5183035000,22:00,11836,-165.0


In [228]:
stay_0901_trip_time['temp'] = stay_0901_trip_time['diff'].fillna(value=stay_0901_trip_time['stay_cnts']).map(lambda x: max(x,0)) 
stay_0901_trip_time

,hdong_cd,time,stay_cnts,diff,temp
0,1111051500,08:00,19621,NaN,19621.0
1,1111051500,09:00,23280,3659.0,3659.0
2,1111051500,10:00,27712,4432.0,4432.0
3,1111051500,11:00,31436,3724.0,3724.0
4,1111051500,12:00,35029,3593.0,3593.0
...,...,...,...,...,...
56443,5183035000,19:00,12378,-164.0,0.0
56444,5183035000,20:00,12350,-28.0,0.0
56445,5183035000,21:00,12001,-349.0,0.0
56446,5183035000,22:00,11836,-165.0,0.0


In [229]:
stay_0901_trip_time['temp2'] = stay_0901_trip_time.groupby(['hdong_cd'])['temp'].cumsum()
stay_0901_trip_time.head(20)

,hdong_cd,time,stay_cnts,diff,temp,temp2
0,1111051500,08:00,19621,NaN,19621.0,19621.0
1,1111051500,09:00,23280,3659.0,3659.0,23280.0
2,1111051500,10:00,27712,4432.0,4432.0,27712.0
3,1111051500,11:00,31436,3724.0,3724.0,31436.0
4,1111051500,12:00,35029,3593.0,3593.0,35029.0
5,1111051500,13:00,38260,3231.0,3231.0,38260.0
6,1111051500,14:00,39103,843.0,843.0,39103.0
7,1111051500,15:00,39361,258.0,258.0,39361.0
8,1111051500,16:00,37720,-1641.0,0.0,39361.0
9,1111051500,17:00,31808,-5912.0,0.0,39361.0


In [230]:
stay_0901_trip_time_cumsum = stay_0901_trip_time.groupby(['hdong_cd'])['temp2'].max().sort_values(ascending=False)
stay_0901_trip_time_cumsum


hdong_cd
1156054000    247827.0
4825054000     85986.0
4119080000     74178.0
1171056600     72415.0
4420025300     65196.0
                ...   
4180037000        16.0
4812963000        15.0
4827036000        11.0
5178034000         3.0
4825032000         0.0
Name: temp2, Length: 3528, dtype: float64

In [231]:
stay_0901_hdong = stay_0901_trip.groupby(['hdong_cd'])['stay_cnts'].max().sort_values(ascending=False).reset_index()
stay_0901_hdong

,hdong_cd,stay_cnts
0,1156054000,15001
1,3020054000,11099
2,4139058900,9520
3,2920063700,8990
4,4119080000,8806
...,...,...
3523,4687038000,43
3524,4872038000,43
3525,5178033000,32
3526,5178034000,21


In [232]:
hdong.loc[stay_0901_hdong.head(20).hdong_cd]

,시도명,시군구명,읍면동명,주소
hdong_cd,,,,
1156054000,서울특별시,영등포구,여의동,서울특별시 영등포구 여의동
3020054000,대전광역시,유성구,온천2동,대전광역시 유성구 온천2동
4139058900,경기도,시흥시,정왕본동,경기도 시흥시 정왕본동
2920063700,광주광역시,광산구,수완동,광주광역시 광산구 수완동
4119080000,경기도,부천시,성곡동,경기도 부천시 성곡동
4825054000,경상남도,김해시,내외동,경상남도 김해시 내외동
1144066000,서울특별시,마포구,서교동,서울특별시 마포구 서교동
1141058500,서울특별시,서대문구,신촌동,서울특별시 서대문구 신촌동
4825055000,경상남도,김해시,북부동,경상남도 김해시 북부동


---
### 목적 없는 녀석들만 보기

In [182]:
stay_0901_etc = stay_0901[stay_0901['purpose'] == 4]
stay_0901_etc

,hdong_cd,date,time,gender,age,purpose,stay_cnts
17,4119080000,20231007,11:00,0,3,4,728
33,5013025300,20231007,09:00,0,4,4,741
35,4148032000,20231007,10:00,0,3,4,716
46,1156054000,20231007,19:00,0,0,4,5317
116,3117051000,20231007,11:00,0,4,4,768
...,...,...,...,...,...,...,...
3841051,4615036000,20231007,10:00,1,7,4,6
3841053,5177036000,20231007,13:00,1,1,4,8
3841055,4793033000,20231007,22:00,1,6,4,9
3841058,4313056000,20231007,19:00,0,8,4,7


In [183]:
stay_0901_etc_reduced =  stay_0901_etc.drop(['date', 'purpose'], axis=1).sort_values(by=['hdong_cd', 'time'])#.groupby(
#     ['hdong_cd', 'time', 'gender', 'age']
# )['stay_cnts'].sum().reset_index()

stay_0901_etc_reduced

,hdong_cd,time,gender,age,stay_cnts
115568,1111051500,08:00,1,4,287
380444,1111051500,08:00,1,6,65
549321,1111051500,08:00,0,8,20
891871,1111051500,08:00,0,4,268
943312,1111051500,08:00,1,3,251
...,...,...,...,...,...
2724613,5183035000,23:00,0,1,52
2745343,5183035000,23:00,0,2,46
2791257,5183035000,23:00,1,6,35
2840898,5183035000,23:00,0,7,29


In [ ]:
stay_0901_etc_time = stay_0901_etc_reduced.groupby(['hdong_cd', 'time'])['stay_cnts'].sum().reset_index()
stay_0901_etc_time['diff'] = stay_0901_etc_time.stay_cnts - stay_0901_etc_time.stay_cnts.shift()

,hdong_cd,time,stay_cnts,diff
0,1111051500,08:00,2262,NaN
1,1111051500,09:00,3913,1651.0
2,1111051500,10:00,4472,559.0
3,1111051500,11:00,4877,405.0
4,1111051500,12:00,5322,445.0
...,...,...,...,...
56413,5183035000,19:00,1554,-150.0
56414,5183035000,20:00,1430,-124.0
56415,5183035000,21:00,1313,-117.0
56416,5183035000,22:00,1210,-103.0


In [185]:
stay_0901_etc_hdong = stay_0901_etc.groupby(['hdong_cd'])['stay_cnts'].max().sort_values(ascending=False).reset_index()
stay_0901_etc_hdong

,hdong_cd,stay_cnts
0,1156054000,15001
1,1117064000,6018
2,1117063000,4816
3,1144066000,4556
4,4127361000,4521
...,...,...
3523,4671033500,25
3524,4121051000,19
3525,4681039000,10
3526,5178033000,7


In [186]:
hdong.loc[stay_0901_etc_hdong.head(20).hdong_cd]

,시도명,시군구명,읍면동명,주소
hdong_cd,,,,
1156054000,서울특별시,영등포구,여의동,서울특별시 영등포구 여의동
1117064000,서울특별시,용산구,이촌제2동,서울특별시 용산구 이촌제2동
1117063000,서울특별시,용산구,이촌제1동,서울특별시 용산구 이촌제1동
1144066000,서울특별시,마포구,서교동,서울특별시 마포구 서교동
4127361000,경기도,안산시 단원구,대부동,경기도 안산시 단원구 대부동
4155025000,경기도,안성시,공도읍,경기도 안성시 공도읍
4812531000,경상남도,창원시 마산합포구,구산면,경상남도 창원시 마산합포구 구산면
1159051000,서울특별시,동작구,노량진제1동,서울특별시 동작구 노량진제1동
4146125000,경기도,용인시 처인구,포곡읍,경기도 용인시 처인구 포곡읍


In [117]:
no = set(hdong.dropna(subset=['읍면동명']).index) - set(stay_0901_trip_time['hdong_cd'].unique())

In [129]:
wait_what = set(stay_0901_trip_time['hdong_cd'].unique()) - set(hdong.dropna(subset=['읍면동명']).index) 
wait_what

{4139065000}

In [127]:
what = hdong.loc[list(no),['읍면동명']].map(lambda x: not x.endswith('출장소'))
what.sum()

읍면동명    1
dtype: int64

In [ ]:
%reset